In [1]:
import rasterio
from rasterio.plot import show
from rasterio.mask import mask
from rasterio.windows import from_bounds
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
import numpy as np
from patchify import patchify
import torch
from torch.functional import F
from torch import nn
from torch.utils.data import DataLoader
from shapely.geometry import box
from torchvision import transforms
import time
import random

from pytorch_segmentation.data.rwanda_dataset import RwandaDataset
from pytorch_segmentation.data.inmemory_dataset import InMemorySatDataset
from pytorch_segmentation.train_net import train
from pytorch_segmentation.models import UNet
import pytorch_segmentation.augmentation.transforms as seg_transforms

seed = 42
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

In [2]:
model_name = "unet_11_07_2022_070457" 
data_parallel = True

save_dir = "saved_models"

#SA data
data_path = "/home/jovyan/work/satellite_data/tmp/2018.vrt"
label_path_sa_test = "data/datasets/V5/test/SA"

#Rwanda data
shape_path_rw = "data/datasets/V1/rwanda_tree_shapes/training_data_polygons_model_29_v2.shp"
test_data_path_rw = "data/datasets/V5/test/rwanda"

test_patch_size = [256,256,3]
overlap_test = 128

batch_size = 50
nworkers = 10
pin_memory = True

padding = False#True

In [3]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
if str(device) == "cpu":
    pin_memory = False

test_transform = None

test_transform = seg_transforms.Compose([
    #seg_transforms.UnmaskEdges([225,225]),
     #seg_transforms.CLAHE_Norm(),
    #seg_transforms.Add_VDVI()
    ])

# 1.1 Data Collection

In [4]:

test_dataset_sa = InMemorySatDataset(data_file_path=data_path,mask_path=label_path_sa_test,
                             overlap=overlap_test,patch_size=test_patch_size,padding=padding,transform=test_transform)

print("Len Test: ",len(test_dataset_sa))

Len Test:  724


In [5]:
test_dataset_rw = RwandaDataset(dataset_path=None,data_file_path=test_data_path_rw,
                   shape_path=shape_path_rw, overlap=overlap_test,patch_size=test_patch_size,padding=padding,transform=test_transform)

print("Len Test: ",len(test_dataset_rw))

100%|██████████| 25/25 [00:12<00:00,  1.97it/s]


Len Test:  1688


In [6]:
test_dataset = torch.utils.data.ConcatDataset([test_dataset_rw, test_dataset_sa])
test_dl = DataLoader(test_dataset,batch_size=batch_size,num_workers=nworkers,
                     shuffle=False,pin_memory=pin_memory,drop_last=False)

# 2 Model

In [7]:
model_path = save_dir + "/" + model_name +  ".pth"
state_dict = torch.load(model_path,map_location="cpu")

In [8]:


if data_parallel:
    # create new OrderedDict that does not contain `module.`
    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:] # remove `module.`
        new_state_dict[name] = v
    model_path = save_dir + "/" + model_name +  "_new.pth"
    torch.save(new_state_dict,model_path)
    

## 2.1 Advanced Unet

In [ ]:
# Change here to adapt to your data
# n_channels=3 for RGB images 
# n_classes is the number of probabilities you want to get per pixel

net = UNet(n_channels=3, n_classes=2, bilinear=False)
#net= DataParallel(net)

if data_parallel:
    net.load_state_dict(new_state_dict)
else:
    net.load_state_dict(state_dict)
#net.load_state_dict(torch.load(model_path,map_location="cpu"))

net = net.to(device=device)



## 2.2 Pretrained Unet


In [ ]:
import segmentation_models_pytorch as smp

net = smp.Unet(
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=2
)


if data_parallel:
    net.load_state_dict(new_state_dict)
else:
    net.load_state_dict(state_dict)


net = net.to(device=device)

## 3 Validation

In [10]:
net.eval()
output = []
y_test = []

with torch.no_grad(): 
    for i,batch in enumerate(test_dl):
        x = batch["x"].to(device)
        out = net(x)#["out"]
        #out = F.softmax(out,dim=1)
        out = torch.argmax(out,dim=1)
        out = out.cpu().numpy()
        output.append(out)
        y_test.extend(batch["y"].numpy())
    y_pred = torch.as_tensor(np.vstack(output))
    y_test = torch.as_tensor(np.array(y_test))

In [11]:
def iou_score(y_pred,target,absent_score=1.0):
    numerator = torch.sum(y_pred * target,dim=1)  # TP
    denominator = torch.sum(y_pred + target,dim=1) - numerator  # 2TP + FP + FN - TP
    iou = (numerator) / (denominator)
    iou[denominator == 0] = absent_score
    return iou.mean()

def dice_coeff(y_pred, target,absent_score=1.0):
    # F1 = TP / (TP + 0.5 (FP + FN)) = 2TP / (2TP + FP + FN)
    numerator = 2 * torch.sum(y_pred * target,dim=1)  # 2TP
    denominator = torch.sum(y_pred + target,dim=1)  # 2TP + FP + FN
    dice = (numerator) / (denominator)
    dice[denominator == 0] = absent_score
    return dice.mean()


In [12]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_29_06_2022_063800
IoU:  0.67069411277771
Dice score:  0.756680428981781
Accuracy:  0.9545283913612366


# Models

## V5

In [24]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_08_07_2022_120733
IoU:  0.6299372911453247
Dice score:  0.7153642177581787
Accuracy:  0.9519509673118591


In [14]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_09_07_2022_091213
IoU:  0.6079014539718628
Dice score:  0.6967523694038391
Accuracy:  0.9387077689170837


In [12]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_29_06_2022_063800
IoU:  0.67069411277771
Dice score:  0.756680428981781
Accuracy:  0.9545283913612366


## V4

In [12]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_29_06_2022_063800
IoU:  0.6824602484703064
Dice score:  0.7717554569244385
Accuracy:  0.9403427243232727


In [25]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_05_07_2022_092338
IoU:  0.6342746615409851
Dice score:  0.7236443758010864
Accuracy:  0.9284062385559082


In [12]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_04_07_2022_094003
IoU:  0.6387783885002136
Dice score:  0.729936420917511
Accuracy:  0.927946150302887


In [12]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_pretrained_04_07_2022_141315
IoU:  0.6783024668693542
Dice score:  0.7673388719558716
Accuracy:  0.938949704170227


In [13]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_pretrained_04_07_2022_141315
IoU:  0.6783024668693542
Dice score:  0.7673388719558716
Accuracy:  0.938949704170227


In [12]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_24_06_2022_082330
IoU:  0.6615725159645081
Dice score:  0.7537617683410645
Accuracy:  0.9354711771011353


In [12]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_28_06_2022_060037
IoU:  0.6614326238632202
Dice score:  0.7512581944465637
Accuracy:  0.9385855793952942


In [12]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_27_06_2022_091925
IoU:  0.6486105918884277
Dice score:  0.7404710054397583
Accuracy:  0.9343949556350708


In [12]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_25_06_2022_155958
IoU:  0.6480911374092102
Dice score:  0.7394369840621948
Accuracy:  0.9371793866157532


In [13]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_dropout_24_06_2022_112935
IoU:  0.5971380472183228
Dice score:  0.6796640157699585
Accuracy:  0.9256259799003601


In [12]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_23_06_2022_155611
IoU:  0.7964387536048889
Dice score:  0.863918662071228
Accuracy:  0.9613916277885437


In [13]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_16_06_2022_190207
IoU:  0.7564991116523743
Dice score:  0.8263688087463379
Accuracy:  0.9605708718299866


# V3

In [13]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_23_06_2022_155611
IoU:  0.6391757130622864
Dice score:  0.7047642469406128
Accuracy:  0.9544192552566528


In [13]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_16_06_2022_190207
IoU:  0.6387537717819214
Dice score:  0.6955344676971436
Accuracy:  0.9621475338935852


In [13]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_05_05_2022_113034
IoU:  0.5952672958374023
Dice score:  0.6595103144645691
Accuracy:  0.9641584753990173


## V2

In [12]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_20_06_2022_191546
IoU:  0.5250652432441711
Dice score:  0.6007120013237
Accuracy:  0.9516292810440063


## V1

In [13]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_16_06_2022_190207
IoU:  0.566116213798523
Dice score:  0.658390462398529
Accuracy:  0.9544999003410339


In [13]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_20_06_2022_124519
IoU:  0.5042276978492737
Dice score:  0.5929812788963318
Accuracy:  0.948485791683197


In [12]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_20_06_2022_124129
IoU:  0.5082292556762695
Dice score:  0.598351776599884
Accuracy:  0.9469634890556335


In [13]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_19_06_2022_174825
IoU:  0.4608975946903229
Dice score:  0.5462993383407593
Accuracy:  0.9406550526618958


In [12]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_19_06_2022_101139
IoU:  0.5205713510513306
Dice score:  0.6123562455177307
Accuracy:  0.946079671382904


In [13]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_19_06_2022_101139
IoU:  0.537010669708252
Dice score:  0.6275419592857361
Accuracy:  0.945489764213562


In [12]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_19_06_2022_095748
IoU:  0.49590927362442017
Dice score:  0.5872480869293213
Accuracy:  0.9463361501693726


In [12]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_17_06_2022_205231
IoU:  0.48862332105636597
Dice score:  0.578204333782196
Accuracy:  0.9475828409194946


In [12]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_17_06_2022_124956
IoU:  0.48286178708076477
Dice score:  0.5737370848655701
Accuracy:  0.9456601142883301


In [12]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_17_06_2022_082105
IoU:  0.47791990637779236
Dice score:  0.5681964159011841
Accuracy:  0.9438368678092957


In [12]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_17_06_2022_082054
IoU:  0.4992392957210541
Dice score:  0.5876631140708923
Accuracy:  0.9477913975715637


In [12]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)


#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_05_05_2022_113034
IoU:  0.5547717213630676
Dice score:  0.645209789276123
Accuracy:  0.9556285738945007


In [12]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)




#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_15_06_2022_175944
IoU:  0.45879825949668884
Dice score:  0.5451763868331909
Accuracy:  0.9410678744316101


In [18]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)




#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_14_06_2022_143718
IoU:  0.5262002944946289
Dice score:  0.6148598194122314
Accuracy:  0.9428917765617371


In [12]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)




#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_16_06_2022_083705
IoU:  0.5006430149078369
Dice score:  0.5897845029830933
Accuracy:  0.9438526034355164


In [11]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)




#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_09_06_2022_091550
IoU:  0.8330698013305664
Dice score:  0.8749065399169922
Accuracy:  0.9755495190620422


In [11]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)




#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_08_06_2022_123439
IoU:  0.817255973815918
Dice score:  0.8576894998550415
Accuracy:  0.9728814959526062


In [11]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)




#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_07_06_2022_140803
IoU:  0.8471149802207947
Dice score:  0.8872880935668945
Accuracy:  0.9784597754478455


In [11]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)




#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc}}])
df.to_csv(model_path+"_score.csv")

Model:  unet_06_06_2022_131709
IoU:  0.8556468486785889
Dice score:  0.8879135847091675
Accuracy:  0.9806501865386963


## 2.3 Unet  Dropout
https://github.com/zijundeng/pytorch-semantic-segmentation

In [14]:
from pytorch_segmentation.models.unet_dropout import UNet


# Change here to adapt to your data
net = UNet(num_classes=2)

if data_parallel:
    net.load_state_dict(new_state_dict)
else:
    net.load_state_dict(state_dict)

net = net.to(device=device)


/home/jovyan/work/notebooks/satellite_data/pytorch_segmentation/models/unet_dropout.py:84: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(module.weight)
